Standard imports


In [21]:
import torch

from torchsummary import summary

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

#DeXpression model

Image pre-processing block

In [17]:
class block1(nn.Module):
  '''
  Image pre-processing block
  '''
  def __init__(self):
    super(block1, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, stride = 2, padding = 3)
    self.pool  = nn.MaxPool2d(kernel_size = 3, size = 2)
    self.lrn  = nn.LocalResponseNorm(size = 2) #LRN as opposed to BN
  
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.lrn(x)
    return x

Feature extractor block (FeatEx-1)

In [18]:
class block2(nn.Module):
  '''
  First feature extractor block
  '''
  def __init__(self):
    super(block2, self).__init__()
    self.conv2a = nn.Conv2d(in_channels = 64, out_channels = 96, kernel_size = 1)
    self.pool2a = nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1)
    self.conv2b = nn.Conv2d(in_channels = 96, out_channels = 208, kernel_size = 3, padding = 1)
    self.conv2c = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 1)
    self.pool2b = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 0)

  def forward(self, x1):
    #Start from the same input
    x2 = x1
    #Path-1
    x1 = F.relu(self.conv2a(x1))
    x1 = F.relu(self.conv2b(x1))
    #Path-2
    x2 = self.pool2a(x2)
    x2 = F.relu(self.conv2c(x2))
    #Concatenate path-1 and path-2
    x3 = torch.cat(x1, x2)
    x3 = self.pool2b(x3)
    return x3


Feature extractor block (FeatExt-2)

In [15]:
class block3(nn.Module):
  '''
   Second feature extractor block 
  '''
  def __init__(self):
    super(block3, self).__init__()
    self.conv3a = nn.Conv2d(in_channels = 208, out_channels = 96, kernel_size = 1)
    self.pool3a = nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1)
    self.conv3b = nn.Conv2d(in_channels = 96, out_channels = 208, kernel_size = 3, stride = 1, padding = 1)
    self.conv3c = nn.Conv2d(in_channels = 208, out_channels = 64, kernel_size = 1)
    self.pool3b = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 0)
    #Dropout regularization
    self.dropout = nn.Dropout(p = 0.125)
    #Fully connected layers (3-layered neural network)
    self.fc1 = nn.Linear(282*14*14*4, 1024)
    self.fc2 = nn.Linear(1024, 64)
    self.fc3 = nn.Linear(64, 8)

  def forward(self, x1):
    #Start from the same input
    x2 = x1
    #Path-1
    x1 = F.relu(self.conv3a(x1))
    x1 = F.relu(self.conv3b(x2))
    #Path-2
    x2 = self.pool3a(x2)
    x2 = F.relu(self.conv3c(x2))
    x2 = F.pad(x2, (0, 0, 0, 0, 72, 72))
    #Concatenate path-1 and path-2
    x3 = torch.cat(x1, x2)
    x3 = self.pool3b(x3)
    x3 = x3.view(-1, 208*13*13*4)
    x3 = F.relu(self.fc1(x3))
    x3 = F.relu(self.fc2(x3))
    x3 = F.log_softmax(self.fc3(x3))
    return x3


Some initial setup

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Hyperparameters
epochs = 100
batch_size = 8
learning_rate = 0.001
